8) El dataframe (sales) lista las ventas de productos con los siguientes campos: Dia, Mes, Año, ProductID, Import(USD). Para un mismo dia, mes y año puede venderse N veces el mismo producto. Por otro lado, tenemos una descripcion de los productos en el dataframe Products: ProductID, Title, Category, Description. Categorie puede ser "men", "women", "kids". Proponer un programa en Pandas que permita:
**A. Indicar los titulos de los productos de la categoria "Men" para los cuales el importe de venta supera el promedo mensual de ventas de los productos de la misma categoria (por ejemplo si el promedio de Abril de "men" es 120dls, y un producto se vendio en abril a 135 dls, debemos listar ese producto). Usar "TRANSFORM"
B. Indicar el top10 de productos que se vendieron mayor cantidad de dias de forma consecutiva**

In [2]:
import pandas as pd
import numpy as np
import random

def random_dates(start, end, n=10):
    start_u = start.value//10**9
    end_u = end.value//10**9
    return pd.to_datetime(np.random.randint(start_u, end_u, n), unit='s')

fechaInicial = pd.to_datetime('2019-01-01')
fechaFinal = pd.to_datetime('2019-02-01')

dataFrameSales = pd.DataFrame(columns = ['Dia','Mes','Año','ProductID','Import(USD)'])
dataFrameProducts = pd.DataFrame(columns = ['ProductID','Title','Category','Description'])

for i in range(30):
    fecha = random_dates(fechaInicial, fechaFinal,1)[0]
    
    dataFrameSales = dataFrameSales.append({
        'Dia' :  fecha.day,
        'Mes' : fecha.month,
        'Año' : fecha.year,
        'ProductID' : random.randint(1,10),
        'Import(USD)' : random.uniform(0,500)
    }, ignore_index = True)

for j in range(1,11):
    dataFrameProducts = dataFrameProducts.append({
        'ProductID' : j,
        'Title' : "Title's product {}".format(j),
        'Category' : random.choice(['Men','Women','Kids']),
        'Description' : "Description's product {}".format(j)        
    }, ignore_index = True)
    
dataFrameSales.Dia = dataFrameSales.Dia.astype(int)
dataFrameSales.Mes = dataFrameSales.Mes.astype(int)
dataFrameSales.Año = dataFrameSales.Año.astype(int)
dataFrameSales.ProductID = dataFrameSales.ProductID.astype(int)
dataFrameSales['Import(USD)'] = pd.to_numeric(dataFrameSales['Import(USD)'])

In [3]:
dataFrameSales

,Dia,Mes,Año,ProductID,Import(USD)
0,17,1,2019,2,77.211176
1,23,1,2019,10,216.549083
2,25,1,2019,6,285.250060
3,19,1,2019,5,419.468855
4,6,1,2019,4,301.687380
5,21,1,2019,3,140.350825
6,29,1,2019,4,191.789949
7,10,1,2019,4,475.849283
8,13,1,2019,3,441.688820
9,19,1,2019,9,56.081588


In [4]:
dataFrameProducts

,ProductID,Title,Category,Description
0,1,Title's product 1,Men,Description's product 1
1,2,Title's product 2,Men,Description's product 2
2,3,Title's product 3,Women,Description's product 3
3,4,Title's product 4,Kids,Description's product 4
4,5,Title's product 5,Women,Description's product 5
5,6,Title's product 6,Kids,Description's product 6
6,7,Title's product 7,Kids,Description's product 7
7,8,Title's product 8,Kids,Description's product 8
8,9,Title's product 9,Men,Description's product 9
9,10,Title's product 10,Kids,Description's product 10


**A. Indicar los titulos de los productos de la categoria "Men" para los cuales el importe de venta supera el promedo mensual de ventas de los productos de la misma categoria (por ejemplo si el promedio de Abril de "men" es 120dls, y un producto se vendio en abril a 135 dls, debemos listar ese producto). Usar "TRANSFORM"**

Transform produce un DataFram con la misma longitud de axis que el dataframe que invoca

In [5]:
productosCategoriaMen = dataFrameProducts[dataFrameProducts['Category'] == 'Men']
ventasMen = productosCategoriaMen.merge(dataFrameSales, on = 'ProductID')

#agrego columna con promedio de ventas mensual
ventasMen['PromedioMensual'] = ventasMen.groupby('Mes')['Import(USD)'].transform('mean')

#me quedo con los que superan el promedio mensual
ventasMen[ventasMen['Import(USD)'] > ventasMen['PromedioMensual']].Title.to_frame()


,Title
0,Title's product 1
1,Title's product 1
2,Title's product 1
5,Title's product 2


**B. Indicar el top10 de productos que se vendieron mayor cantidad de dias de forma consecutiva**

In [35]:
#creo una columna "Fecha" con la fecha q se hizo la compra
newDf = dataFrameSales[['Año','Mes','Dia','ProductID']]
newDf['Fecha'] = newDf['Año'].astype(str) + '-' + newDf['Mes'].astype(str) + '-' + newDf['Dia'].astype(str)
newDf['Fecha'] = pd.to_datetime(newDf['Fecha'])

#ordeno por fecha y producto
newDf.sort_values(by = ['ProductID','Fecha'], inplace = True)

#agrego columna de fecha siguiente
newDf['Fecha siguiente'] = newDf['Fecha'].shift(-1) #1 corrimiento hacia arriba
newDf = newDf.reset_index(drop = True)

#elimino la ultima fila ya que no tiene fecha siguiente (NaT)
b = newDf.groupby('ProductID').apply(lambda x: x.index.max())
newDf = newDf.drop(b)
newDf

,Año,Mes,Dia,ProductID,Fecha,Fecha siguiente
0,2019,1,15,1,2019-01-15,2019-01-27
1,2019,1,27,1,2019-01-27,2019-01-28
3,2019,1,2,2,2019-01-02,2019-01-17
4,2019,1,17,2,2019-01-17,2019-01-25
5,2019,1,25,2,2019-01-25,2019-01-27
6,2019,1,27,2,2019-01-27,2019-01-31
8,2019,1,13,3,2019-01-13,2019-01-21
10,2019,1,1,4,2019-01-01,2019-01-05
11,2019,1,5,4,2019-01-05,2019-01-06
12,2019,1,6,4,2019-01-06,2019-01-09


In [37]:
#diferencia entre fechas
newDf['Diferencia dias'] = newDf['Fecha siguiente'] - newDf['Fecha']
newDf = newDf[newDf['Diferencia dias'] == "1 days"]
newDf

#NO SE COMO SEGUIR

,Año,Mes,Dia,ProductID,Fecha,Fecha siguiente,Diferencia dias
1,2019,1,27,1,2019-01-27,2019-01-28,1 days
11,2019,1,5,4,2019-01-05,2019-01-06,1 days
13,2019,1,9,4,2019-01-09,2019-01-10,1 days
18,2019,1,19,5,2019-01-19,2019-01-20,1 days
20,2019,1,30,5,2019-01-30,2019-01-31,1 days
27,2019,1,22,10,2019-01-22,2019-01-23,1 days
